In [1]:
!pip install torch-scatter==2.0.2 -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.2-cp37-cp37m-linux_x86_64.whl size=7756361 sha256=eb8824bc5018e7c9bdab770a16143c36d5bca6ce7424966f66e32bd6716de51f
  Stored in directory: /root/.cache/pip/wheels/68/94/3a/203b4e701bab9db29f54b9462e62de45e54b0bf63b4ea5ced6
Successfully built torch-scatter


In [2]:
! rm -r transformers
! git clone https://github.com/huggingface/transformers.git
! cd transformers
! pip install ./transformers

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 86905, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 86905 (delta 3), reused 2 (delta 0), pack-reused 86894
Receiving objects: 100% (86905/86905), 69.89 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (62534/62534), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895

# Prepare Data

In [2]:
import pickle
all_claims = pickle.load(open('../../data/corona/corona_allClaims','rb'))
user_claims = pickle.load(open('../../data/corona/user_claims.pkl','rb'))
all_tables = pickle.load(open('../../data/corona/corona_tables','rb'))
ground_truth = pickle.load(open('../../data/corona/corona_GT.pkl','rb'))

In [3]:
import pandas as pd

corona_tables = []
header = ['id','info']

i = 0
for t in all_tables:
  r = ' '.join(t)

  corona_tables.append([i, ' '.join(t)])
  i += 1

df = pd.DataFrame(corona_tables,columns=header)
df.to_csv('corona.csv',index=False)


In [4]:
import os
#os.mkdir('tables')
chunks = [corona_tables[x:x+1000] for x in range(0, len(corona_tables), 1000)]
row_address = {}
for i in range(0,len(chunks)):
  df = pd.DataFrame(chunks[i],columns=header)
  df.to_csv(f'table_{i}.csv',index=False)

  for r in chunks[i]:
    row_address[r[1]] = (f'table_{i}.csv', chunks[i].index(r)   ) 

In [5]:
header = ['question','table_file','answer_coordinates','answer_text']
table = []

for q in ground_truth:
  if len(ground_truth[q]) == 0 : continue
  #if q not in user_claims: continue
  for t in ground_truth[q]:
    ind = row_address[t][1]
    row = [q]
    cor,ans = [f'({ind},{1})'],[t]

    tfile = '' + row_address[t][0]

    row.append(tfile)
  
    row.append(cor)
    row.append(ans)

    table.append(row)


df = pd.DataFrame(table,columns=header)

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)

train.to_csv('queries_train.csv',index=False)
test.to_csv('queries_test.csv',index=False)

## Prepare the data 

Let's look at the first few rows of the dataset:

In [8]:
import pandas as pd
data = pd.read_csv("queries_train.csv")
data.head()

,question,table_file,answer_coordinates,answer_text
0,the total death in russia is below 10000.,table_11.csv,"['(5,1)']",['total deaths russia 0 0 17 1073 4693 5031']
1,death count in france is 120 persons,table_10.csv,"['(27,1)']",['total deaths france 0 2 3532 24379 28805 289...
2,the death rate in america is much higher than ...,table_4.csv,"['(48,1)']",['total confirmed france 5 100 52827 167299 18...
3,qatar has 1000 cases so far.,table_5.csv,"['(24,1)']",['total confirmed qatar 0 1 781 13409 56910 60...
4,there are more than 1000 cases in france,table_4.csv,"['(48,1)']",['total confirmed france 5 100 52827 167299 18...


In [9]:
import ast

def _parse_answer_coordinates(answer_coordinate_str):
  """Parses the answer_coordinates of a question.
  Args:
    answer_coordinate_str: A string representation of a Python list of tuple
      strings.
      For example: "['(1, 4)','(1, 3)', ...]"
  """

  try:
    answer_coordinates = []
    # make a list of strings
    coords = ast.literal_eval(answer_coordinate_str)
    # parse each string as a tuple
    for row_index, column_index in sorted(
        ast.literal_eval(coord) for coord in coords):
      answer_coordinates.append((row_index, column_index))
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_coordinate_str)
  
  return answer_coordinates


def _parse_answer_text(answer_text):
  """Populates the answer_texts field of `answer` by parsing `answer_text`.
  Args:
    answer_text: A string representation of a Python list of strings.
      For example: "[u'test', u'hello', ...]"
    answer: an Answer object.
  """
  try:
    answer = []
    for value in ast.literal_eval(answer_text):
      answer.append(value)
  except SyntaxError:
    raise ValueError('Unable to evaluate %s' % answer_text)

  return answer

data['answer_coordinates'] = data['answer_coordinates'].apply(lambda coords_str: _parse_answer_coordinates(coords_str))
data['answer_text'] = data['answer_text'].apply(lambda txt: _parse_answer_text(txt))

data.head(10)

,question,table_file,answer_coordinates,answer_text
0,the total death in russia is below 10000.,table_11.csv,"[(5, 1)]",[total deaths russia 0 0 17 1073 4693 5031]
1,death count in france is 120 persons,table_10.csv,"[(27, 1)]",[total deaths france 0 2 3532 24379 28805 28943]
2,the death rate in america is much higher than ...,table_4.csv,"[(48, 1)]",[total confirmed france 5 100 52827 167299 189...
3,qatar has 1000 cases so far.,table_5.csv,"[(24, 1)]",[total confirmed qatar 0 1 781 13409 56910 60259]
4,there are more than 1000 cases in france,table_4.csv,"[(48, 1)]",[total confirmed france 5 100 52827 167299 189...
5,the number of cases in france is higher than i...,table_4.csv,"[(52, 1)]",[total confirmed germany 5 79 71808 163009 183...
6,the number of deaths in italy is 8000,table_10.csv,"[(50, 1)]",[total deaths italy 0 29 12428 27967 33415 33530]
7,france has the most confirmed cases in europe.,table_4.csv,"[(48, 1)]",[total confirmed france 5 100 52827 167299 189...
8,death number in iran is higher than the us,table_11.csv,"[(39, 1)]",[total deaths us 0 1 5367 62996 104381 106180]
9,the death rate in the world is 20%,table_5.csv,"[(74, 1)]",[total confirmed world 9927 86011 857608 32569...


In [46]:
data = data.sort_values(by=['answer_coordinates'], ascending=True)
data.head()

,question,table_file,answer_coordinates,answer_text
34,there are 200000 deaths in china.,table_10.csv,"[(1, 1)]",[total deaths china 213 2837 3309 4637 4638 4638]
15,there are 30 cases in morocco,table_5.csv,"[(4, 1)]",[total confirmed morocco 0 0 617 4423 7807 7866]
8,the total death in russia is below 10000.,table_11.csv,"[(5, 1)]",[total deaths russia 0 0 17 1073 4693 5031]
13,the death rate in america more than china,table_4.csv,"[(22, 1)]",[total confirmed china 9802 79356 82279 83956 ...
28,there are 30 cases in colombia,table_4.csv,"[(23, 1)]",[total confirmed colombia 0 0 906 6507 27219 3...


In [10]:
import torch
from transformers import TapasTokenizer
table_csv_path = ''

# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base",drop_rows_to_fit=True)

In [48]:
table_csv_path = ''
item = data.iloc[0]

table = pd.read_csv(table_csv_path + item.table_file).astype(str) 

display(table)
print(item.question)

encoding = tokenizer(table=table, queries=item.question, answer_coordinates=item.answer_coordinates, answer_text=item.answer_text,
                     truncation=True, padding="max_length", return_tensors="pt")
encoding.keys()

,id,info
0,1000,total deaths chile 0 0 12 227 1054 1188
1,1001,total deaths china 213 2837 3309 4637 4638 4638
2,1002,total deaths colombia 0 0 16 293 916 1014
3,1003,total deaths comoros 0 0 0 0 2 2
4,1004,total deaths congo (brazzaville) 0 0 0 9 20 20
...,...,...
95,1095,total deaths pakistan 0 0 26 385 1483 1621
96,1096,total deaths panama 0 0 30 188 336 352
97,1097,total deaths papua new guinea 0 0 0 0 0 0
98,1098,total deaths paraguay 0 0 3 10 11 11


there are 200000 deaths in china.


dict_keys(['input_ids', 'attention_mask', 'labels', 'numeric_values', 'numeric_values_scale', 'token_type_ids'])

In [49]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table = pd.read_csv(table_csv_path + item.table_file).astype(str) # TapasTokenizer expects the table data to be text only
        encoding = self.tokenizer(table=table,
                                   queries=item.question,
                                   answer_coordinates=item.answer_coordinates,
                                   answer_text=item.answer_text,
                                   truncation=True,
                                   padding="max_length",
                                   return_tensors="pt"
         )

          # use encodings of second table-question pair in the batch
        encoding = {key: val[-1] for key, val in encoding.items()}

        #encoding["float_answer"] = torch.tensor(item.float_answer)



        return encoding

    def __len__(self):
        return len(self.df)

#data = pd.read_csv('data/queries.csv')
train_dataset = TableDataset(df=data, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

In [50]:
train_dataset[0]["token_type_ids"].shape

torch.Size([512, 7])

In [ ]:
train_dataset[1]["input_ids"].shape

torch.Size([512])

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
batch["input_ids"].shape

torch.Size([8, 512])

In [ ]:
batch["token_type_ids"].shape

torch.Size([8, 512, 7])

Let's decode the first table-question pair:

In [ ]:
tokenizer.decode(batch["input_ids"][0])

'[CLS] there are 1 new confirmed cases in afghanistan in february. [SEP] id info 0 new 1 new 2 new 3 new 4 new 5 new 6 new 7 new 8 new 9 new 10 new 11 new 12 new 13 new 14 new 15 new 16 new 17 new 18 new 19 new 20 new 21 new 22 new 23 new 24 new 25 new 26 new 27 new 28 new 29 new 30 new 31 new 32 new 33 new 34 new 35 new 36 new 37 new 38 new 39 new 40 new 41 new 42 new 43 new 44 new 45 new 46 new 47 new 48 new 49 new 50 new 51 new 52 new 53 new 54 new 55 new 56 new 57 new 58 new 59 new 60 new 61 new 62 new 63 new 64 new 65 new 66 new 67 new 68 new 69 new 70 new 71 new 72 new 73 new 74 new 75 new 76 new 77 new 78 new 79 new 80 new 81 new 82 new 83 new 84 new 85 new 86 new 87 new 88 new 89 new 90 new 91 new 92 new 93 new 94 new 95 new 96 new 97 new 98 new 99 new 100 new 101 new 102 new 103 new 104 new 105 new 106 new 107 new 108 new 109 new 110 new 111 new 112 new 113 new 114 new 115 new 116 new 117 new 118 new 119 new 120 new 121 new 122 new 123 new 124 new 125 new 126 new 127 new 128 n

In [ ]:
#first example should not have any prev_labels set
assert batch["token_type_ids"][0][:,3].sum() == 0

Let's decode the second table-question pair and verify some more:

In [ ]:
tokenizer.decode(batch["input_ids"][1])

'[CLS] there are 1 new cases in afghanistan in february. [SEP] id info 0 new 1 new 2 new 3 new 4 new 5 new 6 new 7 new 8 new 9 new 10 new 11 new 12 new 13 new 14 new 15 new 16 new 17 new 18 new 19 new 20 new 21 new 22 new 23 new 24 new 25 new 26 new 27 new 28 new 29 new 30 new 31 new 32 new 33 new 34 new 35 new 36 new 37 new 38 new 39 new 40 new 41 new 42 new 43 new 44 new 45 new 46 new 47 new 48 new 49 new 50 new 51 new 52 new 53 new 54 new 55 new 56 new 57 new 58 new 59 new 60 new 61 new 62 new 63 new 64 new 65 new 66 new 67 new 68 new 69 new 70 new 71 new 72 new 73 new 74 new 75 new 76 new 77 new 78 new 79 new 80 new 81 new 82 new 83 new 84 new 85 new 86 new 87 new 88 new 89 new 90 new 91 new 92 new 93 new 94 new 95 new 96 new 97 new 98 new 99 new 100 new 101 new 102 new 103 new 104 new 105 new 106 new 107 new 108 new 109 new 110 new 111 new 112 new 113 new 114 new 115 new 116 new 117 new 118 new 119 new 120 new 121 new 122 new 123 new 124 new 125 new 126 new 127 new 128 new 129 new

In [ ]:
for id, prev_label in zip(batch["input_ids"][1], batch["token_type_ids"][1][:,3]):
  if id != 0:
    print(tokenizer.decode([id]), prev_label.item())

## Define the model

Here we initialize the model with a pre-trained base and randomly initialized cell selection head, and move it to the GPU (if available).

Note that the `google/tapas-base` checkpoint has (by default) an SQA configuration, so we don't need to specify any additional hyperparameters.

In [51]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['output_weights', 'column_output_bias', 'column_output_weights', 'output_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)


## Training the model

Let's fine-tune the model in well-known PyTorch fashion:

In [52]:
import time

st = time.time()
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

Epoch: 0
Loss: 1.7603199481964111
Loss: 0.5257381200790405
Loss: 0.3227989375591278
Loss: 0.12141665816307068
Loss: 0.10102341324090958
Loss: 0.11608704924583435


In [21]:
print(time.time() - st)

5000.853292942047


In [54]:
model.save_pretrained("tapas.mdl")

## Inference


In [1]:
from transformers import TapasForQuestionAnswering

model = TapasForQuestionAnswering.from_pretrained("tapas.mdl")
device='cuda'

In [11]:
test = pd.read_csv("queries_test.csv")

In [12]:
from tqdm import tqdm
preds_tapas = {}

for i in tqdm(range(0,len(test)),position=0):
  item = test.iloc[i]
  queries = [item['question']]
  #if queries[0] not in user_claims: continue
  table = pd.read_csv(item['table_file']).astype(str)
  inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
  outputs = model(**inputs)
  predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(
          inputs,
          outputs.logits.detach()
          
  )
  preds_tapas[item['question']] = [table.iloc[p[0]]['info'] for p in predicted_answer_coordinates[0][0]]
  #print(predicted_answer_coordinates)

100%|██████████| 20/20 [02:17<00:00,  6.89s/it]


In [13]:
########################## METRICS

def HAS_POSITIVE(actual,preds):
    for i in range(0,len(preds)):
        if preds[i] in actual:
            return 1
    return 0

def MRRR(actual,preds):
    for i in range(0,len(preds)):
        if preds[i] in actual:
            return 1/(i+1)
    return 0


def MAP_K(actual,preds):
    precision = 0
    hit = 0
    for i in range(0,len(preds)):
        if preds[i] in actual:
            hit += 1
            precision += hit/(i+1)
    return precision/len(actual)    


In [28]:
for KK in [1,5,20]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for claim in preds_tapas:
        #if len(preds_tapas[claim]) == 0 or claim not in ground_truth: continue
        i+=1
        preds =  preds_tapas[claim] [0:KK]
        golds = [f for f in ground_truth[claim]]

        MAP += MAP_K(golds,preds)
        MRR += MRRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)



#################### 1 ###########################

MRR: 0.05263157894736842 MAP: 0.05263157894736842 HAS POSITIVE: 0.05263157894736842

#################### 5 ###########################

MRR: 0.05263157894736842 MAP: 0.05263157894736842 HAS POSITIVE: 0.05263157894736842

#################### 20 ###########################

MRR: 0.09114131482552536 MAP: 0.07654231733179102 HAS POSITIVE: 0.47368421052631576
